In [1]:
!pip install transformers
!pip install pymorphy3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 59.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=487d50aaa8957f7523856a20e283c60a763d815e14f4a24f07f738c2df6cfc1b
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
!python -m spacy download ru_core_news_sm
!python -m spacy download ru_core_news_md

2023-09-03 16:40:43.187660: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-03 16:40:45.208584: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 28.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
2023-09-03 16:41:16.048024: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-03 16:41:17.772209: W tensorflow/compiler/tf2tensorrt/utils/p

In [3]:
import os
import re
import nltk
import json
import spacy
import pymorphy3
import numpy as np
import transformers
import pandas as pd
from tqdm import tqdm
import ru_core_news_md
import ru_core_news_sm
from pprint import pprint
from spacy import displacy
from transformers import pipeline
from nltk.tokenize import word_tokenize

morph = pymorphy3.MorphAnalyzer()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
COLAB = '/content/drive/MyDrive/sarawan_data/'
DATA = COLAB + 'data/'
PHOTO = COLAB + 'data/photo/'
OBJS = COLAB + 'objs/'
MODELS = COLAB + 'models/'
nlp = spacy.load('ru_core_news_sm')

### Google colab

In [5]:
# colab drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# download photos
import zipfile
with zipfile.ZipFile(PHOTO + 'auchan_perekrestok.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

ALL_PHOTOS = 'auchan_perekrestok'

In [7]:
# load stop_words and datasets on google colab
stop_words = []
with open(DATA + 'utils/stop_words.txt', encoding='utf-8') as f:
     for i in f.readlines():
        stop_words.append(i[:-1])

a = pd.read_json(DATA + 'original/auchan.json')
p = pd.read_json(DATA + 'original/perek.json')

# Обработка данных

### Обработка сырых датасетов

In [8]:
def is_exception(a, b):
    """вытаскиваем имена файлов фоток"""
    try:
        url = a[b]

        if isinstance(url, list):
            url = url[0]

        url = url.split('/')
        return url[-1]
    except:
        return np.nan


def unpack_perekrestok_attributes_dict(attributes_column:pd.Series):
    """вытаскиваем вложенные друг в друга словари атрибутов"""
    attributes_column_list = []
    for i in range(len(attributes_column)):
        info = attributes_column[i]['Информация']
        try:
            features = attributes_column[i]['Свойства товара']
            info.update(features)
        except: pass

        volume = attributes_column[i]['volume']
        info['volume'] = volume
        attributes_column_list.append(info)

    return attributes_column_list

In [9]:
perek = pd.DataFrame()
auchan = pd.DataFrame()

# берем важные колонки
auchan['name'] = a['product_name']
auchan['image'] = a['product_img_links']
auchan['volume'] = a['product_human_volume']
auchan['attributes'] = a['specification']
auchan['cat1'] = a['main_category_name']
auchan['cat2'] = a['category_name']

perek['name'] = p['product_name']
perek['image'] = p['product_img_link']
perek['volume'] = p['product_volume'].apply(lambda x: str(x) + ' ') + p['product_volume_type']
perek['attributes'] = p['features']
perek['cat1'] = p['main_category_name']
perek['cat2'] = p['primary_category']
perek['cat3'] = p['category_name']


# добавляем в атрибуты volume
for i in range(len(perek)):
    perek.loc[i, 'attributes']['volume'] =  perek.loc[i, 'volume']

for i in range(len(auchan)):
    auchan.loc[i, 'attributes']['volume'] =  auchan.loc[i, 'volume']

perek.drop(columns=['volume'], inplace=True)
auchan.drop(columns=['volume'], inplace=True)

# достаем вложенные словари
perek['attributes'] = unpack_perekrestok_attributes_dict(perek['attributes'])

# вытаскиваем имена файлов фоток
auchan['image'] = auchan['image'].apply(lambda x:  is_exception(x, 'preview_url'))
perek['image'] = perek['image'].apply(lambda x:  is_exception(x, 'small_url'))

### Приведение имён и атрибутов к хорошему виду

In [10]:
def each_attribute_pocessing(product):
    """очищает атрибуты товара от атрибутов bad_parameters"""
    bad_parameters = ['рекомендац', 'срок', 'гаранти', 'измерени', 'цена','условия', 'хранения',
                      'isbn', 'дополнени', 'регион', 'время варки', 'не содержит', 'температур',
                      'подача', 'может содержать', 'адрес', 'сочетание', 'тип потребления', 'углевод',
                      'стандарт', 'предосторожност', 'гарантия', 'артикул товара', 'калорийность',
                      'аннотац', 'органы', 'погрешность', 'сторона', 'сертифик', 'cпособ', 'дополнени', 'дополнит', 'бренд', 'состав']
    bad_parameters_for_values = ['да', 'нет', '-']

    attr_names = list(product.keys())
    attr_values = list(product.values())
    brand = ''
    for name, val in zip(attr_names, attr_values):
        flag = 0
        for p in bad_parameters:
            if 'бренд' in name.lower():
                brand = val.lower()
            if p in name.lower():
                del product[name]
                flag = 1
                break

        if flag == 0:
            if val.lower() in bad_parameters_for_values:
                del product[name]
                flag = 1
    return product, brand


def attributes_processing(attr_column: pd.Series):
    """убирает плохие атрибуты, приводит к надлежащему виду атрибуты"""
    attr = attr_column.copy(deep=True)
    brand = attr_column.copy(deep=True)
    for idx in tqdm(attr.index):
        attr[idx], brand[idx] = each_attribute_pocessing(attr[idx])
        attr[idx] = ' '.join(attr[idx].values())
        attr[idx] = ' '.join([morph.parse(i)[0].normal_form.lower() for i in re.findall("[a-zA-Zа-яА-Яё]+|[0-9.%]+", attr[idx]) if i != '' and i not in '.,:;\"\'@!#№$^?*\(\)\\/' and i.lower() not in stop_words])
    return attr, brand

In [11]:
auchan['attributes_prepared'], auchan['brand'] = attributes_processing(auchan['attributes'])
perek['attributes_prepared'], perek['brand'] = attributes_processing(perek['attributes'])

100%|██████████| 12300/12300 [00:28<00:00, 436.80it/s]


### Убираем пропуски в атрибутах

In [22]:
# вспомогательная функция проверки на число.
def is_num(num:str):
  try:
    float(num)
    return True
  except ValueError:
    return False


def clean_attributes(data:str, clean_all=False):
  clean_list = ['вес', 'змж', 'вт','г','кг','л', 'мл', 'лист','листа',\
               'листов', 'элементов', 'элемент', 'файлами', 'файлов',\
               'м','мл', 'мм','наб','пара', 'лет', 'см','упак', 'цв',\
               'цвет', 'цветов', 'цвета', 'шт', 'kg', 'g', 'mg', 'ml', 'l']

  data = data.split()
  # убираем лишние пробелы
  data = [i.strip() for i in data]
  # приклеиваем весовые / обьемные характеристики к их числовым выражениям
  volume_features = {}
  for ind, el in enumerate(data):
    if ind < len(data) - 1:
      if is_num(el) and data[ind + 1] in clean_list:
        new = el + data[ind + 1]
        data.pop(ind)
        data.insert(ind, new)
        data.pop(ind + 1)
        if new in [k for k, v in volume_features.items()]:
          volume_features[new].append(ind)
        else:
            volume_features[new] = [ind]
  # выбрасываем повторы характеристик (например мл мл - один приклеится к числу, другой останется висеть)
  data = [i for i in data if i not in clean_list]
  # здесь убираются повторы только в весовых характеристиках, вроде 1л 1л
  # не знаю, стоит ли убирать вообще все повторы - некоторые из них извлекаются
  # и из имени, и из атрибутов (вроде "масло подсолнечный". - имя и "фракция подсолнечный" - состав)
  # не потеряем-ли часть информации?

  for k, v in volume_features.items():
    if len(v) > 1:
      for index in v[1:]:
        data.pop(index)
  # здесь убираем случайно залетевшие одиночные символы (d, e из консервантов), сохраняя числа
  for el in data:
    if not is_num(el) and len(el) < 2:
      data.remove(el)

  if clean_all:
    final_data = []
    for el in data:
      if is_num(el):
        final_data.append(el)
      elif el not in final_data:
        final_data.append(el)

    return ' '.join(final_data)
  else:
    return ' '.join(data)



def atribute_duplicate_cleaning(dataframe, clean_all=False):
  for i in tqdm(range(len(dataframe))):
    dataframe.loc[i, 'attributes_prepared'] = clean_attributes(dataframe.loc[i, 'attributes_prepared'], clean_all)

  return dataframe


In [25]:
auchan = atribute_duplicate_cleaning(auchan, True)
perek = atribute_duplicate_cleaning(perek, True)

100%|██████████| 12300/12300 [00:03<00:00, 3892.79it/s]


### Подготовка имён

In [37]:
def name_preprocessing(data):
    """приводит имена к надлежащему виду"""
    symbols = ['вес', 'змж', 'вт','г','кг','л', 'мл', 'лист','листа', 'листов', 'элементов', 'элемент', 'файлами', 'файлов', 'м','мл', 'мм','наб','пара', 'лет', 'см','упак', 'цв', 'цвет', 'цветов', 'цвета', 'шт', 'предмета', 'месяцев','месяца', 'год', 'лет', 'сорт', 'вида', 'вид', 'тон', 'смотреть', 'размер', 'без', 'сахара', 'сахар',
               'caramello','daino','nero','айворя','алый','ассорти','бежевый','белые','белый','берюзовый','бирюзовый','бледно','бордо','бордовый','бронза','бронзовый','бурый','голубой','дымка','дымчатый','жёлтый','зелёный','золотистый','золотой','изумрудный','капучиный','карамель','комбинированный','коралл','коралловый',
               'коричневый', 'шоколадный', 'коричнеый','кость','кофейный','красный','кремовый','лаванда','лиловый','лимонный','малиновый','медный','металлик','многоцветный','молочный','морской','мультиколор','мультицвет','мятный','натуральный',
               'нефрит','оливковый','оранжевый','ореховый','оттенок','охра','пастельный','персиковый','прозрачный','пудровыя','пурпурный','разгоцветный','разноцвентный','разноцветеый','разноцветный','разноцветь','рисунок','розовый','рубиновый','салатовый','светло','светлый','серебристый','серебряный','серо','серый','синий',
               'сиреневый','слоновый','сталь','стальной','телесный','темно','терракотовый','тёмный','узор','фиолетовый','фисташковый','фуксия','хаки','хром','цвет','цветной','черно','чёрный','шампаня','янтарный','ярко','white','black','red','yellow', 'orange','green','blue','purple','pink','brown','grey',
               'xxxxl', 'xxxl', 'xxl', 'xl', 'l','xxxxs', 'xxxs', 'xxs', 'xs', 's', 'xxxxm', 'xxxm', 'xxm', 'xm', 'm', 'холодный', 'теплый', 'тёплый', 'газированный', 'негазированный']

    for idx in tqdm(data.index):
        name = ' '
        for word in word_tokenize(data['name'][idx]):
            word = morph.parse(word.replace(',', '.'))[0].normal_form.lower()
            if word not in stop_words and word not in ' \-\|\(\)\.,;:@\"\'\`~$^&*_-+=|/\\\+\!"\'«»':
                name += ' ' + word
                if word in symbols or bool(re.search(r'\d', word)):
                    data['attributes_prepared'][idx] += ' ' + word
        data.loc[idx, 'name_prepared'] = name.strip()
    return data

In [38]:
auchan = name_preprocessing(auchan)
perek = name_preprocessing(perek)

100%|██████████| 12300/12300 [00:44<00:00, 274.35it/s]


### Соединение магазинов

In [51]:
data_concat = pd.concat((auchan, perek), axis=0, ignore_index=True)
data_concat.head()

,name,image,attributes,cat1,cat2,attributes_prepared,brand,name_prepared,cat3
0,"Торт Kristof клубника со сливками, 450 г",small_1000d62a27c1704e71336b046d4e4f3838e85f75...,"{'Вид упаковки': 'Пластик', 'Тип торта': 'Ягод...",Сладости,Торты,пластик ягодный 4.4 19 0.45 0.499 160 210 50 о...,kristof,торт kristof клубника сливка 450 г,NaN
1,"Набор одноразовых стаканов Actuel пластик, 20х...",small_4ed8d12281a9102b9777887b51b814195a190945...,"{'Масса нетто, кг': '0.2', 'Масса брутто, кг':...",Кухня,"Стаканы, бокалы",0.2 0.2 71 117 310 россия 0.2кг 20х200 мл,actuel,набор одноразовый стакан actuel пластик 20х200 мл,NaN
2,"Сироп «Баринофф» десертный Вишня, 1 л",small_4abc7e4195076a6538d2ef5e84c89bcca4b73702...,"{'Упаковка': 'Стеклянная бутылка', 'Тип товара...",Сладости,Сиропы,стеклянный бутылка сироп вишнёвый 1 1.2 85 85 ...,баринофф,сироп баринофф десертный вишня 1 л,NaN
3,Пирожное «Медвежонок Барни» бисквитное с молоч...,small_7c4033e6867cdcfbdb3a847dc97d2401f06ce2d5...,"{'Белки на 100 г, г': '6', 'Жиры на 100 г, г':...",Сладости,"Пирожные, десерты",6 15 0.15 0.16 30 280 95 монядэлиса русьооо ро...,барни,пирожное медвежонок барни бисквитный молочный ...,NaN
4,Пирожное «Медвежонок Барни» бисквитное со сгущ...,small_bdbe36876182b6fd32dc9e0989ecfa0fef23baa1...,"{'Белки на 100 г, г': '6', 'Жиры на 100 г, г':...",Сладости,"Пирожные, десерты",6 15 0.15 0.16 30 300 95 монядэлиса русьооо ро...,барни,пирожное медвежонок барни бисквитный сгустить ...,NaN


In [40]:
for i, j in zip(data_concat['name'], data_concat['name_prepared']):
  print(i, '|', j)

Выходные данные были обрезаны до нескольких последних строк (5000).
Вино Ikhalto Алазанская Долина белое полусладкое 11.5%, 750мл | вино ikhalto алазанский долина белый полусладкое 11.5 % 750мл
Вино Одиши белое полусладкое 11-13%, 750мл | вино одиш белый полусладкое 11-13 % 750мл
Вино Herrenstein Grüner Veltliner белое полусухое 12%, 1л | вино herrenstein grüner veltliner белый полусухой 12 % 1л
Вино Villa Krim Winery Yard белое полусухое 12%, 750мл | вино villa krim winery yard белый полусухой 12 % 750мл
Вино Ionos White белое сухое 11.5%, 750мл | вино ionos white белый сухой 11.5 % 750мл
Вино Фанагория Номерной Резерв Шардоне белое сухое 14%, 750мл | вино фанагорий номерной резерв шардон белый сухой 14 % 750мл
Вино La Casada Pinot Grigio Delle Venezie DOC белое сухое 13%, 750мл | вино la casada pinot grigio delle venezie doc белый сухой 13 % 750мл
Вино Tour De Mandelotte Bordeaux AOC Blanc белое полусладкое 13%, 750мл | вино tour de mandelotte bordeaux aoc blanc белый полусладкое 13 

### Добавление информации о цветах, вкусах и т.д в атрибуты

In [52]:
def extract_attr_pos_tagging(data):
    features = []
    for index, name, attr, brand in tqdm(zip(data.index, data.name, data.attributes, data.brand)):
        name = ' '.join(re.split(r'\W', name.lower()))
        try:
            if brand in name.lower():
                name = name.lower().replace(brand.lower(), brand.title())
        except KeyError:
            pass


        ner = nlp(name)
        attr = []
        for token in ner:
            if (token.pos_ in ('NOUN', 'NUM', 'VERB', 'ADV', 'PROPN', 'X') and token.dep_ not in ('ROOT', 'nsubj') or (token.pos_ == 'VERB' and token.dep_ == 'ROOT') or token.pos_ == 'ADJ') and str(token).istitle() == False and morph.parse(str(token))[0].normal_form not in data['attributes_prepared'][index]:
                attr.append(morph.parse(str(token))[0].normal_form)
        data['attributes_prepared'][index] = data['attributes_prepared'][index] + ' ' + ' '.join(attr)
    return data

In [53]:
data_concat['before_pos_tagging'] = data_concat['attributes_prepared']
data_concat = extract_attr_pos_tagging(data_concat)
data_concat.head(5)

35627it [11:44, 50.58it/s]


,name,image,attributes,cat1,cat2,attributes_prepared,brand,name_prepared,cat3,before_pos_tagging
0,"Торт Kristof клубника со сливками, 450 г",small_1000d62a27c1704e71336b046d4e4f3838e85f75...,"{'Вид упаковки': 'Пластик', 'Тип торта': 'Ягод...",Сладости,Торты,пластик ягодный 4.4 19 0.45 0.499 160 210 50 о...,kristof,торт kristof клубника сливка 450 г,NaN,пластик ягодный 4.4 19 0.45 0.499 160 210 50 о...
1,"Набор одноразовых стаканов Actuel пластик, 20х...",small_4ed8d12281a9102b9777887b51b814195a190945...,"{'Масса нетто, кг': '0.2', 'Масса брутто, кг':...",Кухня,"Стаканы, бокалы",0.2 0.2 71 117 310 россия 0.2кг 20х200 мл одно...,actuel,набор одноразовый стакан actuel пластик 20х200 мл,NaN,0.2 0.2 71 117 310 россия 0.2кг 20х200 мл
2,"Сироп «Баринофф» десертный Вишня, 1 л",small_4abc7e4195076a6538d2ef5e84c89bcca4b73702...,"{'Упаковка': 'Стеклянная бутылка', 'Тип товара...",Сладости,Сиропы,стеклянный бутылка сироп вишнёвый 1 1.2 85 85 ...,баринофф,сироп баринофф десертный вишня 1 л,NaN,стеклянный бутылка сироп вишнёвый 1 1.2 85 85 ...
3,Пирожное «Медвежонок Барни» бисквитное с молоч...,small_7c4033e6867cdcfbdb3a847dc97d2401f06ce2d5...,"{'Белки на 100 г, г': '6', 'Жиры на 100 г, г':...",Сладости,"Пирожные, десерты",6 15 0.15 0.16 30 280 95 монядэлиса русьооо ро...,барни,пирожное медвежонок барни бисквитный молочный ...,NaN,6 15 0.15 0.16 30 280 95 монядэлиса русьооо ро...
4,Пирожное «Медвежонок Барни» бисквитное со сгущ...,small_bdbe36876182b6fd32dc9e0989ecfa0fef23baa1...,"{'Белки на 100 г, г': '6', 'Жиры на 100 г, г':...",Сладости,"Пирожные, десерты",6 15 0.15 0.16 30 300 95 монядэлиса русьооо ро...,барни,пирожное медвежонок барни бисквитный сгустить ...,NaN,6 15 0.15 0.16 30 300 95 монядэлиса русьооо ро...


In [55]:
# до и после pos_tagging
for k, i, j in zip(data_concat['name'], data_concat['before_pos_tagging'], data_concat['attributes_prepared']):
  print(f'Имя:{k} \nДо: {i}\nПосле:  {j}\n\n')

Выходные данные были обрезаны до нескольких последних строк (5000).
Имя:Гель-крем для душа Palmolive Натурэль женский Мягкий и сладкий Персик с увлажняющим молочком, 250мл 
До: colgate palmolive гель душа весь тип турция женщина персик 260.0g 250мл
После:  colgate palmolive гель душа весь тип турция женщина персик 260.0g 250мл крем натурэль женский мягкий сладкий увлажнять молочко


Имя:Гель для душа Palmolive MEN мужской Спорт Восстанавливающий 3в1 для тела лица и волос, 250мл 
До: colgate palmolive гель душа турция весь тип мужчина свежий 260.0g 3в1 250мл
После:  colgate palmolive гель душа турция весь тип мужчина свежий 260.0g 3в1 250мл men мужской спорт восстанавливать тело лицо волос


Имя:Гель для душа Synergetic Сандал и ягоды можжевельника натуральный биоразлагаемый, 380мл 
До: ооо синергетика гель душа весь тип россия ягода 400.0g натуральный 380мл
После:  ооо синергетика гель душа весь тип россия ягода 400.0g натуральный 380мл сандал можжевельник биоразлагать


Имя:Гель-уход 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Выходные данные были обрезаны до нескольких последних строк (5000).
Имя:Пиво Жигули ИПА светлое нефильтрованное 4.5%, 450мл 
До: московский пивоваренный компания россия 5 горький нефильтрованный 0.45 светлый пастеризовать 450.0g светлый 4.5 450мл
После:  московский пивоваренный компания россия 5 горький нефильтрованный 0.45 светлый пастеризовать 450.0g светлый 4.5 450мл ипа


Имя:Напиток пивной Чехов вишнёвый эль пастеризованный нефильтрованный 6.3%, 750мл 
До: василеостровский пивоварня россия фрукт пастеризовать 750.0g 6.3 750мл
После:  василеостровский пивоварня россия фрукт пастеризовать 750.0g 6.3 750мл пивной вишнёвый эль нефильтрованный


Имя:Пиво Охота крепкое светлое фильтрованное 8.1%, 1.25л 
До: россия фильтровать хейнекен пастеризовать 8.100 горький хмельный 1250.0g светлый 8.1 1.25ть
После:  россия фильтровать хейнекен пастеризовать 8.100 горький хмельный 1250.0g светлый 8.1 1.25ть крепкий 25л


Имя:Пиво Brew Moose ИПА светлое нефильтрованное пастеризованное 5.8%, 450мл 
Д

In [ ]:
# old version
'''def each_attribute_pocessing(product):
    """очищает атрибуты товара от атрибутов bad_parameters"""
    bad_parameters = ['рекомендац', 'срок', 'гаранти', 'измерени', 'цена','условия', 'хранения',
                      'isbn', 'дополнени', 'регион', 'время варки', 'не содержит', 'температур',
                      'подача', 'может содержать', 'адрес', 'сочетание', 'тип потребления', 'углевод',
                      'стандарт', 'предосторожност', 'гарантия', 'артикул товара', 'калорийность',
                      'аннотац', 'органы', 'погрешность', 'сторона', 'сертифик', 'cпособ', 'дополнени', 'дополнит', 'бренд']
    bad_parameters_for_values = ['да', 'нет', '-']

    # параметры, которые не должны удаляться из названий продуктов
    common_parameters = ['тип', 'вид', 'модель', 'производитель', 'серия', 'состав', 'бренд', 'тип продукта', 'тип товара', 'вид продукта', 'комплектац']
    common_attributes = []


    attr_names = list(product.keys())
    attr_values = list(product.values())

    for name, val in zip(attr_names, attr_values):
        flag = 0
        for p in bad_parameters:
            if p in name.lower():
                del product[name]
                flag = 1
                break

        if flag == 0:
            if val.lower() in bad_parameters_for_values:
                del product[name]
                flag = 1

        if flag == 0:
            for common_parameter in common_parameters:
                if common_parameter in name.lower():
                    for word in re.split(r'[ \-\|\(\),."\'«»]', product[name]):
                        common_attributes.append(morph.parse(word)[0].normal_form.lower())
                    break

    common_attributes = ' '.join(common_attributes)
    return product, common_attributes


def attributes_processing(attr_column: pd.Series):
    """убирает плохие атрибуты, приводит к надлежащему виду атрибуты"""
    attr = attr_column.copy(deep=True)
    common_attr = pd.Series(index=range(len(attr)), dtype='object').fillna('')
    for idx in tqdm(attr.index):
        attr[idx], common_attr[idx] = each_attribute_pocessing(attr[idx])
        attr[idx] = ' '.join(attr[idx].values())
        attr[idx] = ' '.join([morph.parse(i)[0].normal_form.lower() for i in re.findall("[a-zA-Zа-яА-Яё]+|[0-9.%]+", attr[idx]) if i != '' and i not in '.,:;\"\'@!#№$^?*\(\)\\/' and i.lower() not in stop_words])
    return attr, common_attr

auchan['attributes_prepared'], auchan['common_attributes'] = attributes_processing(auchan['attributes'])
perek['attributes_prepared'], perek['common_attributes'] = attributes_processing(perek['attributes'])'''

'def each_attribute_pocessing(product):\n    """очищает атрибуты товара от атрибутов bad_parameters"""\n    bad_parameters = [\'рекомендац\', \'срок\', \'гаранти\', \'измерени\', \'цена\',\'условия\', \'хранения\',\n                      \'isbn\', \'дополнени\', \'регион\', \'время варки\', \'не содержит\', \'температур\',\n                      \'подача\', \'может содержать\', \'адрес\', \'сочетание\', \'тип потребления\', \'углевод\',\n                      \'стандарт\', \'предосторожност\', \'гарантия\', \'артикул товара\', \'калорийность\',\n                      \'аннотац\', \'органы\', \'погрешность\', \'сторона\', \'сертифик\', \'cпособ\', \'дополнени\', \'дополнит\', \'бренд\']\n    bad_parameters_for_values = [\'да\', \'нет\', \'-\']\n    \n    # параметры, которые не должны удаляться из названий продуктов\n    common_parameters = [\'тип\', \'вид\', \'модель\', \'производитель\', \'серия\', \'состав\', \'бренд\', \'тип продукта\', \'тип товара\', \'вид продукта\', \'комплектац

In [ ]:
# old version
'''def split(data):
    """убирает атрибуты из имён, приводит имена к надлежащему виду"""
    symbols = ['вес', 'змж', 'вт','г','кг','л', 'мл', 'лист','листа', 'листов', 'элементов', 'элемент', 'файлами', 'файлов', 'м','мл', 'мм','наб','пара', 'лет', 'см','упак', 'цв', 'цвет', 'цветов', 'цвета', 'шт', 'предмета', 'месяцев','месяца', 'год', 'лет', 'сорт', 'вида', 'вид', 'тон', 'смотреть', 'размер', 'без', 'сахара', 'сахар',
               'caramello','daino','nero','айворя','алый','ассорти','бежевый','белые','белый','берюзовый','бирюзовый','бледно','бордо','бордовый','бронза','бронзовый','бурый','голубой','дымка','дымчатый','жёлтый','зелёный','золотистый','золотой','изумрудный','капучиный','карамель','комбинированный','коралл','коралловый',
               'коричневый', 'шоколадный', 'коричнеый','кость','кофейный','красный','кремовый','лаванда','лиловый','лимонный','малиновый','медный','металлик','многоцветный','молочный','морской','мультиколор','мультицвет','мятный','натуральный',
               'нефрит','оливковый','оранжевый','ореховый','оттенок','охра','пастельный','персиковый','прозрачный','пудровыя','пурпурный','разгоцветный','разноцвентный','разноцветеый','разноцветный','разноцветь','рисунок','розовый','рубиновый','салатовый','светло','светлый','серебристый','серебряный','серо','серый','синий',
               'сиреневый','слоновый','сталь','стальной','телесный','темно','терракотовый','тёмный','узор','фиолетовый','фисташковый','фуксия','хаки','хром','цвет','цветной','черно','чёрный','шампаня','янтарный','ярко','white','black','red','yellow', 'orange','green','blue','purple','pink','brown','grey',
               'xxxxl', 'xxxl', 'xxl', 'xl', 'l','xxxxs', 'xxxs', 'xxs', 'xs', 's', 'xxxxm', 'xxxm', 'xxm', 'xm', 'm', 'холодный', 'теплый', 'тёплый', 'газированный', 'негазированный']

    for idx in tqdm(data.index):
        name = ''
        for word in re.split(r'[ \-\|\(\)%,\.\+\!"\'«»]', data['name'][idx]):
            word = morph.parse(word)[0].normal_form.lower()
            if re.findall(r'[0-9]+', word) or word in symbols:
                data['attributes_prepared'][idx] += ' ' + word
            elif (word not in word_tokenize(data['attributes_prepared'][idx]) or word in word_tokenize(data['common_attributes'][idx])) and word not in symbols and word.isnumeric() == False and word not in stop_words:
                name += word + ' '
        data.loc[idx, 'name_prepared'] = " ".join(name.split())
    return data

auchan = split(auchan)
perek = split(perek)'''

### Проверка валидности фоток

In [56]:
def clear_images(dataframe, photo_dir='all_photos'):
    """убирает из датафрейма товары, если их фоток нет в папке photo_dir"""
    image_folder = os.listdir(photo_dir)
    bad_df = pd.DataFrame(columns = dataframe.columns)
    print(dataframe.shape)

    for i in range(len(dataframe)):
        try:
            if not dataframe.loc[i, 'image'] in image_folder:
                bad_df.loc[i, :] = dataframe.loc[i, :]
        except:
            print(dataframe.loc[i, 'image'])
            break

    dataframe.drop(index=bad_df.index, inplace=True)
    dataframe.reset_index(drop=True, inplace=True)
    print(dataframe.shape)
    return dataframe

In [57]:
data_concat = clear_images(data_concat)
data_concat.reset_index(drop=True, inplace=True)

(35627, 10)
(35603, 10)


In [ ]:
len(set(data_concat['cat1']))

27

### Обработка категорий

In [ ]:
data_concat = data_concat.replace({'cat1' : {'Алкоголь (самовывоз)' : 'Алкогольные напитки',
                                             'Соки, воды, напитки' : 'Вода, соки, напитки',
                                             'Кофе, чай, какао, сахар' : 'Чай, кофе, горячие напитки',
                                             'Шоколад, конфеты, сладости' : 'Сладости',
                                             'Чипсы, снеки, попкорн' : 'Снеки',
                                             'Чипсы, снеки, сухофрукты' : 'Снеки',
                                             'Орехи, семечки, сухофрукты' : 'Снеки',
                                             'Овощи, фрукты, грибы' : 'Овощи, фрукты, ягоды, грибы',
                                             'Овощи, фрукты, орехи' : 'Овощи, фрукты, ягоды, грибы',
                                             'Уборка и бытовая химия' : 'Уборка, товары для дома',
                                             'Уборка' : 'Уборка, товары для дома',
                                             'Товары для дома' : 'Уборка, товары для дома',
                                             'Для дома и дачи' : 'Уборка, товары для дома',
                                             'Рыба' : 'Морепродукты',
                                             'Рыба, икра, морепродукты' : 'Морепродукты',
                                             'Красота и гигиена' : 'Красота, гигиена, аптека',
                                             'Отдых на даче' : 'Сад и огород',
                                             'Молоко, сыр, яйца' : 'Молочные продукты',
                                             'Молочные продукты, яйца' : 'Молочные продукты',
                                             'От Перекрёстка' : 'Собственные торговые марки',
                                             'Посуда' : 'Кухня',
                                             'Все для детей' : 'Игрушки',
                                             'Сухие завтраки, мюсли' :'Макароны, крупы',
                                             'Макароны, крупы, масло, специи' : 'Макароны, крупы',
                                             'Продукты быстрого приготовления' : 'Готовая еда'}})

In [ ]:
data_concat = data_concat[(data_concat['cat1'] != 'Зоотовары') &
                          (data_concat['cat1'] != 'Игрушки') &
                          (data_concat['cat1'] != 'Канцтовары') &
                          (data_concat['cat1'] != 'Книги, журналы') &
                          (data_concat['cat1'] != 'Сад и огород') &
                          (data_concat['cat1'] != 'Системы нагревания, табак') &
                          (data_concat['cat1'] != 'Спорт и отдых') &
                          (data_concat['cat1'] != 'Сад и огород')]

In [ ]:
data_concat.head(2)

,name,image,attributes,cat1,cat2,attributes_prepared,brand,name_prepared,cat3
0,"Торт Kristof клубника со сливками, 450 г",small_1000d62a27c1704e71336b046d4e4f3838e85f75...,"{'Вид упаковки': 'Пластик', 'Тип торта': 'Ягод...",Сладости,Торты,пластик ягодный 4.4 19 0.45 0.499 молоко крем ...,kristof,торт kristof клубника сливка 450 г,NaN
1,"Набор одноразовых стаканов Actuel пластик, 20х...",small_4ed8d12281a9102b9777887b51b814195a190945...,"{'Масса нетто, кг': '0.2', 'Масса брутто, кг':...",Кухня,"Стаканы, бокалы",0.2 0.2 71 117 310 россия 0.2 кг 20х200 мл одн...,actuel,набор одноразовый стакан actuel пластик 20х200 мл,NaN


In [ ]:
count_cat = len(set(data_concat['cat1']))
count_cat

27

In [ ]:
for i, j, k in zip(data_concat['name_prepared'], data_concat['attributes_prepared'], data_concat['brand']):
  print(i)
  print(j)
  print(k)
  print()

Выходные данные были обрезаны до нескольких последних строк (5000).

абрикос корзинка 450 г
пакет абрикос 0.9 0.1 0.45 0.475 абрикос 20 40 60 узбекистан турция 0.45 кг 450 г корзинка
-

маракуйя 250 г
пакет маракуйя 2.4 0.4 0.25 0.275 маракуйя 60 40 20 колумбия китай 0.25 кг 250 г 
-

груша форелла
пакет груша 0.4 0.1 1 1.025 груша форелла аргентина юар 1 кг вес
-

мандарин
пакет мандарин 0.8 0.2 1 1.025 мандарин аргентина юар 1 кг вес
-

ягодный микс голубика клубника малина минь 150 г
контейнер ягодный микс 0.3 0.3 0.15 0.175 голубика малина клубника 600 400 200 армения перу 0.15 кг 150 г минь
-

лимон узбекистан 2 шт 1 упаковка 150 г
пакет лимон 1.1 0.3 0.2 0.225 лимон 60 40 20 узбекистан 0.2 кг 2 шт 1 150 г упаковка
-

апельсин сок 0.8-1.2 кг 1 упаковка 1 кг
подложка апельсин 0.9 0.2 1 1.025 апельсин 290 190 120 узбекистан турция 1 кг 0.8-1.2 кг 1 1 кг сок упаковка
-

ягодный микс клубника голубика 500 г
контейнер ягодный микс 0.8 0.4 0.15 0.175 клубника голубика 600 400 200 россия

KeyboardInterrupt: ignored

In [58]:
data_concat.to_json(DATA + 'prepared_data/03_09_23_after_pos_tagging.json')

# Ембеддинги

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from torchvision.io import read_image
from torch.utils.data import Dataset
import torchvision.models as models
from torchvision import transforms
from torch.nn import BatchNorm1d
import torch.nn as nn
import torch
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CNN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.resnet = models.resnet34(pretrained=True)
        self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features, count_cat)

    def forward(self, img):
        embedding = self.get_embedding(img)

        # возвращаем ембеддинг
        return embedding


    def get_embedding(self, img_tensor):
        img_tensor = img_tensor.to(device)
        layer = self.resnet._modules.get('avgpool')
        outputs = []

        def copy_embeddings(m, i, o):
            """Copy embeddings from the penultimate layer.
            """
            o = o[:, :, 0, 0].cpu().detach().numpy().tolist()
            outputs.append(o)
            hook.remove()

        # attach hook to the penulimate layer
        hook = layer.register_forward_hook(copy_embeddings)


        self.resnet.eval()

        self.resnet(img_tensor)
        list_embeddings = [item for sublist in outputs for item in sublist]
        self.embedding = np.array(list_embeddings)
        return self.embedding

In [ ]:
class NameBert(nn.Module):
    '''Implement embedding from text
    Args:
        text(string): name or attributes
    Returns:
        embeddings(numpy.ndarray): embeddings, shape(batch, seq, emb)
    '''

    def __init__(self, count_cat):
        super().__init__()
        self.m = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2')
        self.m.classifier = torch.nn.Linear(self.m.classifier.in_features, count_cat)

    def forward(self, text):
        # на вход подаётся уже токенизированный текст

        with torch.no_grad():
            model_output = self.m(**{k: v.to(self.m.device) for k, v in text.items()}, output_hidden_states=True)

        """
        embeddings содержит 4 ембеддинга из разных слоёв. Стоит попробовать использовать каждый по отдельности, либо же MaxPooling, AveragePooling, GlobalPooling. нужно тестить , пока оставлю последний слой

        каждый такой эмбеддинг содержит ещё 240 слоёв, тоже можно попробовать пулинг
        """
        embeddings = model_output.hidden_states[-1][:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings.detach()

In [ ]:
class AttributeBert(nn.Module):
    '''Implement embedding from text
    Args:
        text(string): name or attributes
    Returns:
        embeddings(numpy.ndarray): embeddings, shape(batch, seq, emb)
    '''

    def __init__(self):
        super().__init__()
        self.m = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2')
        self.m.classifier = torch.nn.Linear(self.m.classifier.in_features, count_cat)

    def forward(self, text):
        # на вход подаётся уже токенизированный текст

        with torch.no_grad():
            model_output = self.m(**{k: v.to(self.m.device) for k, v in text.items()}, output_hidden_states=True)

        """
        embeddings содержит 4 ембеддинга из разных слоёв. Стоит попробовать использовать каждый по отдельности, либо же MaxPooling, AveragePooling, GlobalPooling. нужно тестить , пока оставлю последний слой

        каждый такой эмбеддинг содержит ещё 240 слоёв, тоже можно попробовать пулинг
        """
        embeddings = model_output.hidden_states[-1][:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings.detach()

In [ ]:
class Embedding(nn.Module):
    def __init__(self):
        super().__init__()


    def forward(self, name_bert, attr_bert, cnn, name, attr, img, num_features, device): #img
        '''
        Args:
            name_bert: экземпляр TinyBert
            attr_bert: экземпляр TinyBert
            cnn: экземпляр CNN
            name: токенизированное название
            attr: токенизированные атрибуты
            img: image из загрузчика трансформированная
            num_features: количество фичей в финальном эмбеддинге
            device: экземпляр девайса cpu или cudo

        Returns:
            embedding: torch.tensor   size = (10, 1136)
        '''


        name = name_bert(name)

        attr = attr_bert(attr)

        img = torch.from_numpy(cnn(img)).to(device)


        self.BN = BatchNorm1d(num_features)
        # embedding_raw = np.hstack((name, attr, img))
        embedding_raw = torch.cat((name, attr, img), dim=1)

        self.BN.to(device, dtype=float)
        embedding = self.BN(embedding_raw.to(device, dtype=float))


        return embedding, attr

### Загрузчик

In [ ]:
class CustomMatchingDataset(Dataset):
    def __init__(self, data, img_dir):
        """
        data.columns:
            - image: название изображения
            - attributes_prepared: строка с атрибутами
            - name_prepared: строка с названием продукта
            - cat1: label с категорией cat1
            - cat2: label с категорией cat2

        """

        # self.le_cat1 = LabelEncoder()
        # self.le_cat2 = LabelEncoder()


        # data['cat1_enc'] = self.le_cat1.fit_transform(data['cat1'].values)
        # data['cat2_enc'] = self.le_cat2.fit_transform(data['cat2'].values)

        self.data = data
        self.images = data['image']
        self.img_dir = img_dir
        self.transform = transforms.Compose([
                                                transforms.ToPILImage(),
                                                transforms.Resize((256, 256)),
                                                transforms.ToTensor()

                                            ])


        self.names = data['name_prepared']
        self.attributes = data['attributes_prepared']
        # self.labels_cat1 = data['cat1_enc']
        # self.labels_cat2 = data['cat2_enc']


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # label_cat1 = self.labels_cat1.iloc[idx]
        # label_cat2 = self.labels_cat2.iloc[idx]


        img_path = os.path.join(self.img_dir, self.images.iloc[idx])
        image = read_image(img_path)
        if self.transform:
            image = self.transform(image).float()

        name = self.names.iloc[idx]
        attribute = self.attributes.iloc[idx]

        return name, attribute, image, idx, #label_cat1, label_cat2

In [ ]:
custom_dataset = CustomMatchingDataset(data=data_concat, img_dir=ALL_PHOTO_DIR)
data_loader = torch.utils.data.DataLoader(custom_dataset, batch_size=100, shuffle=False)

### Load models

In [ ]:
cnn = CNN()
name_bert = NameBert()
attr_bert = AttributeBert()
emb = Embedding()

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
final_embedding_lenght = cnn.resnet.fc.in_features + name_bert.m.classifier.in_features + attr_bert.m.classifier.in_features

name_bert.to(device)
attr_bert.to(device)
cnn.to(device)
emb.to(device)

NameError: name 'CNN' is not defined

In [ ]:
# colab
cnn.resnet.load_state_dict(torch.load(COLAB + "models/cnn/")

name_bert.m.load_state_dict(torch.load(COLAB + "models/names_bert/")

attr_bert.m.load_state_dict(torch.load(COLAB + "models/attr_bert/")

In [ ]:
# without colab
cnn.resnet.load_state_dict(torch.load("data/models/resnet34.pth", map_location=torch.device('cpu')))

name_bert.m.load_state_dict(torch.load("data/models/names_bert.pth", map_location=torch.device('cpu')))

attr_bert.m.load_state_dict(torch.load("data/models/attr_bert.pth", map_location=torch.device('cpu')))  # map_location=torch.device('cpu')

<All keys matched successfully>

In [ ]:
data_with_embeddings = None
for name, attribute, image, index in data_loader:
    """создание ембеддингов"""
    torch.cuda.empty_cache()
    name = tokenizer(name, padding=True, truncation=True, return_tensors='pt')
    attribute = tokenizer(attribute, padding=True, truncation=True, return_tensors='pt')
    emb_main, emb_attr = emb(name_bert, attr_bert, cnn, name.to(device), attribute.to(device), image.to(device), final_embedding_lenght, device)
    emb_main = emb_main.cpu().detach().numpy()
    emb_attr = emb_attr.cpu().detach().numpy()

    data = pd.DataFrame(data_concat.loc[index.cpu().detach().numpy(), :])

    if data_with_embeddings is None:
        data['emb_main'] = [emb_main[i] for i in range(len(emb_main))]
        data['emb_attr'] = [emb_attr[i] for i in range(len(emb_attr))]
        data_with_embeddings = data
    else:
        data['emb_main'] = [emb_main[i] for i in range(len(emb_main))]
        data['emb_attr'] = [emb_attr[i] for i in range(len(emb_attr))]
        data_with_embeddings = pd.concat((data_with_embeddings, data), axis=0)

gc.collect()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│    3 │   """создание ембеддингов"""                                                              │
│    4 │   name = tokenizer(name, padding=True, truncation=True, return_tensors='pt')              │
│    5 │   attribute = tokenizer(attribute, padding=True, truncation=True, return_tensors='pt')    │
│ ❱  6 │   emb_main, emb_attr = emb(name_bert, attr_bert, cnn, name.to(device), attribute.to(de    │
│    7 │   emb_main = emb_main.cpu().detach().numpy()                                              │
│    8 │   emb_attr = emb_attr.cpu().detach().numpy()                                              │
│    9                                                                                             │
│                                                                                                  │
│ C:\Users\Chiki\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module │
│ .py:1194 in _call_impl                                                                           │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ in forward:25                                                                                    │
│                                                                                                  │
│   22 │   │                                                                                       │
│   23 │   │   name = name_bert(name)                                                              │
│   24 │   │                                                                                       │
│ ❱ 25 │   │   attr = attr_bert(attr)                                                              │
│   26 │   │                                                                                       │
│   27 │   │   img = torch.from_numpy(cnn(img)).to(device)                                         │
│   28                                                                                             │
│                                                                                                  │
│ C:\Users\Chiki\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module │
│ .py:1194 in _call_impl                                                                           │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used    

In [ ]:
data_with_embeddings.head()

,name,image,attributes,cat1,cat2,attributes_prepared,common_attributes,name_prepared,cat3,embedding
0,Вино Vinotuar Каберне красное полусладкое Росс...,small_61fdb6d232eccac938289289314c3fa669bfbe11...,"{'Содержание алкоголя, %': '10-12', 'Содержани...",Алкоголь (самовывоз),Красное вино,10 12 полусладкое стеклянный бутылка каберне к...,,вино vinotuar,NaN,"[0.6815848912427855, -0.7642703588393644, -2.6..."
1,Вентилятор настольный Qilive FT30Q белый,small_d37b61afa3cf9667f27f1f8e2269bea7baac20c0...,"{'Масса нетто, кг': '2.7', 'Масса брутто, кг':...",Бытовая техника,Климатическая техника,2.7 3.8 540 210 360 китай 2.7 кг,,вентилятор настольный qilive ft30q,NaN,"[-0.658435963109256, 0.18827417725152046, 1.55..."
2,Увлажнитель Polaris PUH 5305,small_19e49f998a8b7315de449b7a79f24f5f4459c429...,"{'Мощность, Вт': '25', 'Пульт ДУ': 'Нет', 'Мас...",Бытовая техника,Климатическая техника,25 нет 1.13 1.5 340 220 220 китай 4.5 4.5 л,,увлажнитель polaris puh,NaN,"[0.7147956189307539, 0.4064244022641321, 1.715..."
3,"Транспортир Berlingo Flexy гибкий 180°, 10 см",small_d861d349244d89b3d30b44181c1410b66909e58c...,"{'Масса нетто, кг': '0.023', 'Масса брутто, кг...",Канцтовары,Транспортиры,0.023 0.023 130 90 2 китай 0.023 кг,,транспортир berlingo flexy гибкий 180°,NaN,"[-0.03210984472842884, 0.3611008830929113, 0.2..."
4,Насос ручной с иглой,small_8cecd7f70f5853f1de071129909514db611be11b...,"{'Масса нетто, кг': '0.09', 'Масса брутто, кг'...",Спорт и отдых,Инвентарь,0.09 0.1 пластик 250 40 40 ханчжоу кайноо имп ...,,насос ручной игла,NaN,"[-0.4037466701789554, -0.8113269088179401, 0.2..."


In [ ]:
# сохранение данных с ембеддингами
# data_with_embeddings.to_csv('data/data_with_embeddings.csv', index=False)
# data_with_embeddings.to_csv('/content/drive/MyDrive/Colab Notebooks/goods_sarawan/data_with_embeddings.csv', index=False)

# Ембеддинги

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from torchvision.io import read_image
from torch.utils.data import Dataset
import torchvision.models as models
from torchvision import transforms
from torch.nn import BatchNorm1d
import torch.nn as nn
import torch
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CNN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.resnet = models.resnet34(pretrained=True)
        self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features, count_cat)

    def forward(self, img):
        embedding = self.get_embedding(img)

        # возвращаем ембеддинг
        return embedding


    def get_embedding(self, img_tensor):
        img_tensor = img_tensor.to(device)
        layer = self.resnet._modules.get('avgpool')
        outputs = []

        def copy_embeddings(m, i, o):
            """Copy embeddings from the penultimate layer.
            """
            o = o[:, :, 0, 0].cpu().detach().numpy().tolist()
            outputs.append(o)
            hook.remove()

        # attach hook to the penulimate layer
        hook = layer.register_forward_hook(copy_embeddings)


        self.resnet.eval()

        self.resnet(img_tensor)
        list_embeddings = [item for sublist in outputs for item in sublist]
        self.embedding = np.array(list_embeddings)
        return self.embedding

In [ ]:
class NameBert(nn.Module):
    '''Implement embedding from text
    Args:
        text(string): name or attributes
    Returns:
        embeddings(numpy.ndarray): embeddings, shape(batch, seq, emb)
    '''

    def __init__(self):
        super().__init__()
        self.m = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2')
        self.m.classifier = torch.nn.Linear(self.m.classifier.in_features, count_cat)

    def forward(self, text):
        # на вход подаётся уже токенизированный текст

        with torch.no_grad():
            model_output = self.m(**{k: v.to(self.m.device) for k, v in text.items()}, output_hidden_states=True)

        """
        embeddings содержит 4 ембеддинга из разных слоёв. Стоит попробовать использовать каждый по отдельности, либо же MaxPooling, AveragePooling, GlobalPooling. нужно тестить , пока оставлю последний слой

        каждый такой эмбеддинг содержит ещё 240 слоёв, тоже можно попробовать пулинг
        """
        embeddings = model_output.hidden_states[-1][:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings.detach()

In [ ]:
class AttributeBert(nn.Module):
    '''Implement embedding from text
    Args:
        text(string): name or attributes
    Returns:
        embeddings(numpy.ndarray): embeddings, shape(batch, seq, emb)
    '''

    def __init__(self):
        super().__init__()
        self.m = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2')
        self.m.classifier = torch.nn.Linear(self.m.classifier.in_features, count_cat)

    def forward(self, text):
        # на вход подаётся уже токенизированный текст

        with torch.no_grad():
            model_output = self.m(**{k: v.to(self.m.device) for k, v in text.items()}, output_hidden_states=True)

        """
        embeddings содержит 4 ембеддинга из разных слоёв. Стоит попробовать использовать каждый по отдельности, либо же MaxPooling, AveragePooling, GlobalPooling. нужно тестить , пока оставлю последний слой

        каждый такой эмбеддинг содержит ещё 240 слоёв, тоже можно попробовать пулинг
        """
        embeddings = model_output.hidden_states[-1][:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings.detach()

In [ ]:
class Embedding(nn.Module):
    def __init__(self):
        super().__init__()


    def forward(self, name_bert, cnn, name, img, num_features, device): #img
        '''
        Args:
            name_bert: экземпляр TinyBert
            attr_bert: экземпляр TinyBert
            cnn: экземпляр CNN
            name: токенизированное название
            attr: токенизированные атрибуты
            img: image из загрузчика трансформированная
            num_features: количество фичей в финальном эмбеддинге
            device: экземпляр девайса cpu или cudo

        Returns:
            embedding: torch.tensor   size = (10, 1136)
        '''


        name = name_bert(name)

        img = torch.from_numpy(cnn(img)).to(device)


        self.BN = BatchNorm1d(num_features)
        embedding_raw = torch.cat((name, img), dim=1)
        self.BN.to(device, dtype=float)
        embedding = self.BN(embedding_raw.to(device, dtype=float))

        return embedding

### Загрузчик

In [ ]:
class CustomMatchingDataset(Dataset):
    def __init__(self, data, img_dir):
        """
        data.columns:
            - image: название изображения
            - attributes_prepared: строка с атрибутами
            - name_prepared: строка с названием продукта
            - cat1: label с категорией cat1
            - cat2: label с категорией cat2

        """

        # self.le_cat1 = LabelEncoder()
        # self.le_cat2 = LabelEncoder()


        # data['cat1_enc'] = self.le_cat1.fit_transform(data['cat1'].values)
        # data['cat2_enc'] = self.le_cat2.fit_transform(data['cat2'].values)

        self.data = data
        self.images = data['image']
        self.img_dir = img_dir
        self.transform = transforms.Compose([
                                                transforms.ToPILImage(),
                                                transforms.Resize((380, 380)),
                                                transforms.ToTensor()

                                            ])


        self.names = data['name_prepared']
        self.attributes = data['attributes_prepared']
        # self.labels_cat1 = data['cat1_enc']
        # self.labels_cat2 = data['cat2_enc']


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # label_cat1 = self.labels_cat1.iloc[idx]
        # label_cat2 = self.labels_cat2.iloc[idx]


        img_path = os.path.join(self.img_dir, self.images.iloc[idx])
        image = read_image(img_path)
        if self.transform:
            image = self.transform(image).float()

        name = self.names.iloc[idx]
        attribute = self.attributes.iloc[idx]

        return name, attribute, image, idx, #label_cat1, label_cat2

In [ ]:
custom_dataset = CustomMatchingDataset(data=data_concat, img_dir = ALL_PHOTOS)
data_loader = torch.utils.data.DataLoader(custom_dataset, batch_size = 10, shuffle=False)

### Load models

In [ ]:
cnn = CNN()
name_bert = NameBert()
attr_bert = AttributeBert()
emb = Embedding()

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
final_embedding_lenght = cnn.resnet.fc.in_features + name_bert.m.classifier.in_features

name_bert.to(device)
attr_bert.to(device)
cnn.to(device)
emb.to(device)

/home/alfr/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/alfr/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initial

Embedding()

In [ ]:
# colab
cnn.resnet.load_state_dict(torch.load(MODELS + "resnet34.pth"))

name_bert.m.load_state_dict(torch.load(MODELS + "name_bert.pth"))

attr_bert.m.load_state_dict(torch.load(MODELS + "attr_bert.pth"))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3238887572.py, line 2)

In [ ]:
data = None
for name, attribute, image, index in tqdm(data_loader):
    """создание ембеддингов"""
    name = tokenizer(name, padding=True, truncation=True, return_tensors='pt')
    attribute = tokenizer(attribute, padding=True, truncation=True, return_tensors='pt')
    emb_main = emb(name_bert, cnn, name.to(device), image.to(device), final_embedding_lenght, device)
    emb_attr = attr_bert(attribute.to(device))
    emb_main = emb_main.cpu().detach().numpy()
    emb_attr = emb_attr.cpu().detach().numpy()

    d = pd.DataFrame(data_concat.loc[index.cpu().detach().numpy(), :])

    if data is None:
        d['emb_main'] = [emb_main[i] for i in range(len(emb_main))]
        d['emb_attr'] = [emb_attr[i] for i in range(len(emb_attr))]
        data = d
    else:
        d['emb_main'] = [emb_main[i] for i in range(len(emb_main))]
        d['emb_attr'] = [emb_attr[i] for i in range(len(emb_attr))]
        data = pd.concat((data, d), axis=0)

gc.collect()

100%|███████████████████████████████████████| 3179/3179 [03:19<00:00, 15.97it/s]


1917

In [ ]:
data_with_embeddings = data.copy()
data.head(2)

,name,image,attributes,cat1,cat2,attributes_prepared,common_attributes,name_prepared,cat3,emb_main,emb_attr
0,"Торт Kristof клубника со сливками, 450 г",small_1000d62a27c1704e71336b046d4e4f3838e85f75...,"{'Бренд': 'Kristof', 'Вид упаковки': 'Пластик'...",Сладости,Торты,kristof пластик ягодный 4.4 19 0.45 0.499 моло...,kristof пластик ягодный молоко крем маскарпон...,торт kristof клубника сливка,NaN,"[1.9086947188086578, -0.5930325605295548, -0.2...","[-0.00041722265, -0.0065582697, 0.07130568, -0..."
1,"Набор одноразовых стаканов Actuel пластик, 20х...",small_4ed8d12281a9102b9777887b51b814195a190945...,"{'Бренд': 'Actuel', 'Масса нетто, кг': '0.2', ...",Кухня,"Стаканы, бокалы",actuel 0.2 0.2 71 117 310 россия 0.2 кг 20х200 мл,actuel,набор одноразовый стакан actuel пластик,NaN,"[-1.33751321237513, -0.8278393385456816, -1.39...","[0.11178812, 0.074976176, 0.063243695, -0.1229..."


In [ ]:
# сохранение данных с ембеддингами
# data_with_embeddings.to_json(DATA + 'prepared_data/data_with_embeddings.json')

# Сравнение пар

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import ast
import gc

In [ ]:
data_with_embeddings = pd.read_json(DATA + 'prepared_data/data_with_embeddings.json')
data_with_embeddings.head(5)

,name,image,attributes,cat1,cat2,attributes_prepared,common_attributes,name_prepared,cat3,emb_main,emb_attr
0,"Торт Kristof клубника со сливками, 450 г",small_1000d62a27c1704e71336b046d4e4f3838e85f75...,"{'Бренд': 'Kristof', 'Вид упаковки': 'Пластик'...",Сладости,Торты,kristof пластик ягодный 4.4 19 0.45 0.499 моло...,kristof пластик ягодный молоко крем маскарпон...,торт kristof клубника сливка,None,"[1.9086947188, -0.5930325605, -0.2763403084, -...","[-0.0004172226, -0.0065582697, 0.0713056773000..."
1,"Набор одноразовых стаканов Actuel пластик, 20х...",small_4ed8d12281a9102b9777887b51b814195a190945...,"{'Бренд': 'Actuel', 'Масса нетто, кг': '0.2', ...",Кухня,"Стаканы, бокалы",actuel 0.2 0.2 71 117 310 россия 0.2 кг 20х200 мл,actuel,набор одноразовый стакан actuel пластик,None,"[-1.3375132124, -0.8278393385, -1.3910317569, ...","[0.1117881164, 0.074976176, 0.0632436946, -0.1..."
2,"Сироп «Баринофф» десертный Вишня, 1 л",small_4abc7e4195076a6538d2ef5e84c89bcca4b73702...,"{'Бренд': 'Баринофф', 'Упаковка': 'Стеклянная ...",Сладости,Сиропы,баринофф стеклянный бутылка сироп сироп вишнёв...,баринофф сироп сироп сахар вода питьевой сок...,сироп баринофф десертный вишня,None,"[-0.4559286899, -0.5185093988, -1.669124532400...","[0.0500340387, 0.0310449637, 0.076084882, -0.0..."
3,Пирожное «Медвежонок Барни» бисквитное с молоч...,small_7c4033e6867cdcfbdb3a847dc97d2401f06ce2d5...,"{'Бренд': 'Барни', 'Белки на 100 г, г': '6', '...",Сладости,"Пирожные, десерты",барни 6 15 0.15 0.16 мука пшеничный сироп глюк...,барни мука пшеничный сироп глюкозный фруктозн...,пирожное медвежонок барни бисквитный,None,"[-0.0420609734, -0.0379722823, 0.5118518677, 0...","[-0.0056898352, 0.0191023853, 0.0530820675, -0..."
4,Пирожное «Медвежонок Барни» бисквитное со сгущ...,small_bdbe36876182b6fd32dc9e0989ecfa0fef23baa1...,"{'Бренд': 'Барни', 'Белки на 100 г, г': '6', '...",Сладости,"Пирожные, десерты",барни 6 15 0.15 0.16 мука пшеничный сироп глюк...,барни мука пшеничный сироп глюкозный фруктозн...,пирожное медвежонок барни бисквитный сгустить ...,None,"[0.7041283541000001, -0.9771809856, 0.41934521...","[-0.0061359513, 0.0142159043, 0.0491248779, -0..."


## Main embeddings (first level)

In [ ]:
# по категориям
def look_for_identical_products_main(data, threshold):
    """поиск идентичных товаров по основным ембеддингам"""
    count_in_cats = []
    similar_products = []
    cats = list(set(data['cat1']))

    for cat in tqdm(cats):
        count = 0
        indexes = data[data['cat1'] == cat].index
        emb = pd.DataFrame(np.vstack(data.loc[indexes, 'emb_main'].values))
        emb.set_index(indexes, inplace=True)
        similarity = pd.DataFrame(cosine_similarity(emb))
        similarity.set_index(indexes, inplace=True)
        similarity.columns = similarity.index

        for row in indexes:
            for col in indexes:
                if row != col and [similarity[row][col], col, row] not in similar_products and similarity[row][col] >= threshold:
                    similar_products.append([similarity[row][col], row, col])
                    count += 1
        count_in_cats.append([cat, count])
    return similar_products, count_in_cats

In [ ]:
# лучше брать threshold поменьше, лучше лишние на втором уровне отсеять
identical_products, counts = look_for_identical_products_main(data_with_embeddings, 0.7)

NameError: name 'look_for_identical_products_main' is not defined

In [ ]:
# анализируем результат 1-го уровня
# сохраняем идентичные продукты 1-го уровня

count_1 = 0
threshold_1 = 0.7
identical_products_first = []
for sim_main, i, j in identical_products:
    if sim_main > threshold_1:
        identical_products_first.append([sim_main, i, j])
        count_1 += 1
        print(sim_main, '|',  i, data_with_embeddings['name'][i], '|', couple[2], data_with_embeddings['name'][j], '\n')

## Attr embeddings (second level)

In [ ]:
# анализируем результат 2-го уровня
# сохраняем идентичные продукты 2-го уровня

count_2 = 0
threshold_2 = 0.999
identical_products_second = []
for sim_main, i, j in identical_products_first:
    similarity = cosine_similarity(np.array(data_with_embeddings['emb_attr'][i]).reshape(1, -1), np.array(data_with_embeddings['emb_attr'][j]).reshape(1, -1))
    if similarity > threshold_2:
        identical_products_second.append([sim_main, i, j])
        print(similarity, '|', i, data_with_embeddings['name'][i], '|', j, data_with_embeddings['name'][j])

In [ ]:
print(counts)
print('Найденные идентичные продукты на 1-ом уровне с threshold = ' + threshold_1 + ': ' + count_1)
print('Найденные идентичные продукты на 2-ом уровне с threshold = ' + threshold_2 + ': ' + count_2)

In [ ]:
# для анализа результата, можно оценить фотографии найденных идентичных товаров
def show_imgs(data, index1, index2):
    """вывод изображений"""
    plt.figure(figsize=(20, 10))

    photo1 = data.loc[index1, 'image']
    photo2 = data.loc[index2, 'image']

    # Путь к файлам с изображениями
    path_to_image1 = f'{ALL_PHOTOS}{photo1}'
    path_to_image2 = f'{ALL_PHOTOS}{photo2}'

    # Загрузка изображений
    image1 = mpimg.imread(path_to_image1)
    image2 = mpimg.imread(path_to_image2)

    # Создание нового графика
    plt.figure()

    # Первое изображение
    plt.subplot(1, 2, 1)
    plt.imshow(image1)
    plt.title(data.loc[index1, 'name'], fontsize=8)
    plt.axis('off')  # Отключение осей

    # Второе изображение
    plt.subplot(1, 2, 2)
    plt.imshow(image2)
    plt.title(data.loc[index2, 'name'], fontsize=8)
    plt.axis('off')
    plt.show()
    return None

## Поиск идентичных продуктов с помощью алгоритма

In [ ]:
# сейчас что-то подобное на сайте
def compare_names(data):
    similarity = []
    cats = list(set(data['cat1']))

    for cat in tqdm(cats):
        d = data[data['cat1'] == cat]
        indexes = d.index

        for idx, i in enumerate(indexes[:-1]):
            for j in indexes[idx + 1:]:
                if d['name'][i].lower() == d['name'][j].lower():
                    similarity.append([i, j])
    return similarity

In [ ]:
simple_similar_products = compare_names(data_with_embeddings)

In [ ]:
print('Найдено простым алгоритмом: ' + str(len(simple_similar_products)))
print('\nНайденные идентичные продукты на 1-ом уровне с threshold = ' + threshold_1 + ': ' + count_1)
print('\nНайденные идентичные продукты на 2-ом уровне с threshold = ' + threshold_2 + ': ' + count_2)